# steps

1. run get_img_to_yolo_ouput.py.ipynb, get true values (image_to_cells_true.jl)
2. run detect_server_1007/ev.py, get pred values (image_to_cells_pred.jl)
3. iou
4. acc

# 1. test

In [63]:
import joblib as jl

image_to_cells_true = jl.load('image_to_cells_true.jl')
image_to_cells_pred = jl.load('image_to_cells_pred.jl')

In [64]:
type(image_to_cells_true),len(image_to_cells_true),type(image_to_cells_pred),len(image_to_cells_pred)

(dict, 1328, dict, 10)

In [65]:
image_to_cells_pred['0dcca702-a4ef-4fb3-a940-9c0c140b21c7.png'][:5],len(image_to_cells_pred['0dcca702-a4ef-4fb3-a940-9c0c140b21c7.png'])

([(457, 233, 580, 354),
  (473, 370, 603, 490),
  (1009, 317, 1130, 439),
  (1153, 370, 1271, 483),
  (365, 634, 479, 749)],
 312)

In [66]:
image_to_cells_true['0dcca702-a4ef-4fb3-a940-9c0c140b21c7.png'][:5],len(image_to_cells_true['0dcca702-a4ef-4fb3-a940-9c0c140b21c7.png'])

([(455.0, 230.0, 582.0, 352.0),
  (587.0, 818.0, 709.0, 930.0),
  (1006.0, 315.0, 1128.0, 439.0),
  (869.0, 334.0, 994.0, 454.0),
  (364.0, 633.0, 479.0, 751.0)],
 92)

# 2. calculation iou

In [67]:
def get_iou(c1, c2):
    x1_min,y1_min,x1_max,y1_max = c1
    x2_min,y2_min,x2_max,y2_max = c2
    left_column_max = max(x1_min,x2_min)
    right_column_min = min(x1_max,x2_max)
    up_row_max = max(y1_min,y2_min)
    down_row_min = min(y1_max,y2_max)
    if left_column_max>=right_column_min or up_row_max>=down_row_min:
        return 0
        
    area_cross = (right_column_min-left_column_max)*(down_row_min-up_row_max)
    area_s1 = (x1_max-x1_min)*(y1_max-y1_min)
    area_s2= (x2_max-x2_min)*(y2_max-y2_min)
    return area_cross/(area_s1+area_s2-area_cross)

In [68]:
# test
r1=(2,3,10,12)
r2=(12,5,20,24)

print(get_iou(r1,r2))

r1=(2,2,4,4)
r2=(3,3,5,5)
print(get_iou(r1,r2))



0
0.14285714285714285


# 3. iteration each pred

In [69]:
import joblib as jl

def get_correct_pred_cell_count(image_name):
    image_to_cells_true = jl.load('image_to_cells_true.jl')
    image_to_cells_pred = jl.load('image_to_cells_pred.jl')
    cells_pred = image_to_cells_pred[image_name]
    cells_true = image_to_cells_true[image_name]
    
    dup_true_cells = []
    correct_pred_count = 0
    for cell_pred in cells_pred:
        for cell_true in cells_true:
            iou = get_iou(cell_pred, cell_true)
            if iou>0.8 and cell_true not in dup_true_cells:
                correct_pred_count+=1
                dup_true_cells.append(cell_true)
                break
    return correct_pred_count

In [70]:
get_correct_pred_cell_count('0dcca702-a4ef-4fb3-a940-9c0c140b21c7.png')

79

In [71]:
image_to_cells_pred = jl.load('image_to_cells_pred.jl')
all_cell_num_pred = 0
all_cell_num_pred_correct = 0
for image_name in image_to_cells_pred:
    print('processing {0}'.format(image_name))
    cell_num_pred = len(image_to_cells_pred[image_name])
    cell_num_pred_correct = get_correct_pred_cell_count(image_name)
    all_cell_num_pred+=cell_num_pred
    all_cell_num_pred_correct+=cell_num_pred_correct
acc = all_cell_num_pred_correct/all_cell_num_pred

processing 065af549-320a-4862-b92a-8ef750d7b5fb.png
processing 0dcca702-a4ef-4fb3-a940-9c0c140b21c7.png
processing 10be6380-cbbb-4886-8b9e-ff56b1710576.png
processing 13099edb-35d9-438f-b093-2cf2ebf9d255.png
processing 2559636b-f01a-4414-93da-210c3b12d153.png
processing 67acd5bc-4658-4c6e-a9b9-cd8ea3a03418.png
processing 6b14c855-8561-417c-97a4-63fa552842fd.png
processing 7c66d2fd-68ee-4628-b860-9d7ad25b2a80.png
processing 84c88143-940d-4b62-bbd2-7acc4cadcb6b.png
processing 8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png


In [72]:
print(acc)

0.25422138836772984
